SC : https://jesusleal.io/2020/10/20/RoBERTA-Text-Classification/

https://github.com/jlealtru/website_tutorials/blob/main/notebooks/RoBERTA%20with%20IMDB.ipynb

In [1]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import wandb
import os


In [2]:
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'])

In [3]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [4]:
# Display the first few rows of the DataFrame
train_df

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0
...,...,...
24995,A hit at the time but now better categorised a...,1
24996,I love this movie like no other. Another time ...,1
24997,This film and it's sequel Barry Mckenzie holds...,1
24998,'The Adventures Of Barry McKenzie' started lif...,1


In [7]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Load dataset
data = train_df

# Load stopwords
stop_words = set(stopwords.words('english'))

# Preprocessing and tokenization function
def preprocess(text):
    words = word_tokenize(text.lower())  # Tokenization and lowercasing
    words = [word for word in words if word.isalpha() and word not in stop_words]  # Remove non-alphabet and stopwords
    return words

# Separate data based on labels
label_0_data = data[data['label'] == 0]
label_1_data = data[data['label'] == 1]

# Count word frequencies
label_0_words = []
label_1_words = []

for text in label_0_data['text']:
    label_0_words.extend(preprocess(text))
    
for text in label_1_data['text']:
    label_1_words.extend(preprocess(text))

# Calculate word frequencies
label_0_word_freq = Counter(label_0_words)
label_1_word_freq = Counter(label_1_words)

# Get most common words
n_most_common = 10  # You can adjust this value
most_common_label_0 = label_0_word_freq.most_common(n_most_common)
most_common_label_1 = label_1_word_freq.most_common(n_most_common)

print("Most common words for Label 0:", most_common_label_0)
print("Most common words for Label 1:", most_common_label_1)

Most common words for Label 0: [('br', 52636), ('movie', 24097), ('film', 18474), ('one', 12614), ('like', 10967), ('would', 7672), ('even', 7664), ('good', 7206), ('bad', 7139), ('really', 6240)]
Most common words for Label 1: [('br', 49235), ('film', 20284), ('movie', 18498), ('one', 13279), ('like', 8778), ('good', 7452), ('story', 6562), ('great', 6327), ('time', 6051), ('see', 5872)]


In [19]:
# load model and tokenizer and define length of the text sequence
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))


In [21]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


In [22]:
train_data

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [23]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [24]:
train_df

,label,input_ids,attention_mask
0,tensor(0),"[tensor(0), tensor(100), tensor(16425), tensor...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,tensor(0),"[tensor(0), tensor(113), tensor(100), tensor(1...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,tensor(0),"[tensor(0), tensor(1106), tensor(129), tensor(...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
3,tensor(0),"[tensor(0), tensor(713), tensor(822), tensor(2...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
4,tensor(0),"[tensor(0), tensor(7516), tensor(6), tensor(21...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
...,...,...,...
24995,tensor(1),"[tensor(0), tensor(250), tensor(478), tensor(2...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
24996,tensor(1),"[tensor(0), tensor(100), tensor(657), tensor(4...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
24997,tensor(1),"[tensor(0), tensor(713), tensor(822), tensor(8...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
24998,tensor(1),"[tensor(0), tensor(108), tensor(133), tensor(3...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


Trainer Helper Class

In [11]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = 'results',
    num_train_epochs=3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,    
    per_device_eval_batch_size= 8,
    evaluation_strategy = "epoch",
    save_strategy="epoch",              # addition
    disable_tqdm = False, 
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    logging_dir='/media/jlealtru/data_files/github/website_tutorials/logs',
    dataloader_num_workers = 4,
    run_name = 'roberta-classification'
)


In [13]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [15]:
# train the model
trainer.train()


c:\Users\shiro\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


  1%|          | 8/1170 [00:30<50:59,  2.63s/it]  

{'loss': 0.0, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.02}


  1%|▏         | 16/1170 [00:49<47:57,  2.49s/it]

{'loss': 0.0, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.04}


...

100%|██████████| 1170/1170 [1:15:31<00:00,  3.40s/it]c:\Users\shiro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                     
100%|██████████| 1170/1170 [1:22:46<00:00,  3.40s/it]

{'eval_loss': nan, 'eval_accuracy': 0.5, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 432.7112, 'eval_samples_per_second': 57.775, 'eval_steps_per_second': 7.222, 'epoch': 3.0}


100%|██████████| 1170/1170 [1:22:55<00:00,  4.25s/it]

{'train_runtime': 4991.868, 'train_samples_per_second': 15.024, 'train_steps_per_second': 0.234, 'train_loss': 0.0, 'epoch': 3.0}


TrainOutput(global_step=1170, training_loss=0.0, metrics={'train_runtime': 4991.868, 'train_samples_per_second': 15.024, 'train_steps_per_second': 0.234, 'train_loss': 0.0, 'epoch': 3.0})

In [14]:
import wandb
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: faiqfawwazain (rouch). Use `wandb login --relogin` to force relogin


In [16]:
config = wandb.config
config.logging_dir='logs'
config.dataloader_num_workers = 8,

    

In [17]:
# train the model
trainer.train()


c:\Users\shiro\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 8/1170 [00:32<53:18,  2.75s/it]  

{'loss': 0.6984, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.02}


  1%|▏         | 16/1170 [00:51<46:00,  2.39s/it]

{'loss': 0.7011, 'learning_rate': 1.6000000000000001e-06, 'epoch': 0.04}


  2%|▏         | 24/1170 [01:09<43:45,  2.29s/it]

...

                                                     
100%|██████████| 1170/1170 [1:28:39<00:00,  3.41s/it]

{'eval_loss': 0.16553638875484467, 'eval_accuracy': 0.95624, 'eval_f1': 0.9566629694184757, 'eval_precision': 0.9475047080979284, 'eval_recall': 0.966, 'eval_runtime': 443.0918, 'eval_samples_per_second': 56.422, 'eval_steps_per_second': 7.053, 'epoch': 3.0}


100%|██████████| 1170/1170 [1:29:39<00:00,  4.60s/it]

{'train_runtime': 5379.4943, 'train_samples_per_second': 13.942, 'train_steps_per_second': 0.217, 'train_loss': 0.17678481300926616, 'epoch': 3.0}


TrainOutput(global_step=1170, training_loss=0.17678481300926616, metrics={'train_runtime': 5379.4943, 'train_samples_per_second': 13.942, 'train_steps_per_second': 0.217, 'train_loss': 0.17678481300926616, 'epoch': 3.0})

In [18]:
trainer.evaluate()


100%|██████████| 3125/3125 [06:14<00:00,  8.34it/s]


{'eval_loss': 0.13033834099769592,
 'eval_accuracy': 0.9526,
 'eval_f1': 0.9528320662341282,
 'eval_precision': 0.9481898122474848,
 'eval_recall': 0.95752,
 'eval_runtime': 382.5698,
 'eval_samples_per_second': 65.348,
 'eval_steps_per_second': 8.168,
 'epoch': 3.0}